In [1]:
## Part 1: Web scraping with request and beatifulSoup

from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import re


def get_item(bs):
    """
    Obtains url and title from beatifulSoup object.
    """
    x = str(bs.select("a")[0]).split('>')
    url = 'https://english.elpais.com' + x[0].split('"')[1]
    return url

def scrap_news(n_pages):
    """
    Scrap ElPais News-Index website for urls of news. Iterates over a number 'n_pages' of pages.
    """    
    news = []
    for i in range(n_pages):
        url = 'https://english.elpais.com/international/%s/' % i
        r1 = requests.get(url)
        soup = BeautifulSoup(r1.content, "html.parser")    
        coverpage_news = soup.find_all('h2', class_='c_t')    
        news.append(coverpage_news)
    return [get_item(item) for sublist in news for item in sublist]   


def news_content_to_dataframe(list_urls):
    """
    Iterate over a list of url from "El Pais" news website. Extract the content 
    and metadata from each element and save them in pandas Dataframe.
    """
    ds = []
    for url in list_urls:
        r1 = requests.get(url)
        coverpage = r1.content
        soup = BeautifulSoup(coverpage, 'html5lib')
        text = soup.get_text()  
        try:
        # Check by hand the location and structure of the content. 
        # Only works for 'El Pais' Website.
            match = re.findall(r'(?<=]{"@context":).*?(?=:root)', text)[0]
            js = '{"@context":' + match 
            data = json.loads(js)
            ds.append(data)
        # Ignores the small portion of websites with different html structure.
        except IndexError:
            pass
    return pd.DataFrame(ds) 

In [2]:
# Gets a list of urls with news searching in the first 10 pages of the index
urls = scrap_news(10)

In [3]:
# Scraps each url for its content. Then creates a dataframe.
df = news_content_to_dataframe(urls)

In [4]:
# Typical content of the dataframe
df.iloc[0]

@context                                               https://schema.org
@type                                 [NewsArticle, ReportageNewsArticle]
headline                Caro Quintero, the old drug lord who revolutio...
datePublished                                    2022-07-17T23:16:53.566Z
dateModified                                     2022-07-17T23:16:53.566Z
copyrightYear                                                      2022.0
description             At 69, he was the longest-serving drug traffic...
articleBody             “Why are you smiling so much? Has life treated...
keywords                                                               []
author                  [{'@type': 'Person', 'name': 'Jacobo García', ...
articleSection                                              International
mainEntityOfPage        {'@type': ['WebPage', 'ItemPage'], '@id': 'htt...
inLanguage                                                          en-us
contentLocation                      {

In [5]:
# Part 2:  Cleans the dataframe obtained from web scraping news. 
#         This Dataframe will be uploaded into a Postgres DB

import numpy as np

cols = ['@type', 'headline', 'datePublished', 'dateModified', 'description', 'articleBody', 'author', 
        'articleSection','inLanguage', 'contentLocation','copyrightYear','mainEntityOfPage', 
        'publisher','license','isAccessibleForFree', 'isPartOf']
df0 = df[cols]
df0 = df0.rename(columns={'@type':'type', 'headline':'title', 'datePublished':'date_pub', 'dateModified':'date_mod',
                 'copyrightYear':'copyright_year','articleBody':'article_body','articleSection':'category',
                 'mainEntityOfPage':'main_entity', 'inLanguage':'lang', 'contentLocation':'location', 
                  'isAccessibleForFree':'free_access', 'isPartOf':'Part_of'})

# These functions will help to clean the original dataframe
def get_author(row):
    try:
        match = re.findall(r'(?<=\'name\': ).*?(?=,)', str(row.author))
        return match[0].split('\'')[1]
    except IndexError:
        pass
    
def get_location(row):
    try:
        return row.location.get('name')
    except AttributeError:
        pass

def get_entity(row):
    try:
        return row.main_entity.get('@type')[0]
    except AttributeError:
        pass

def get_publisher(row):
    try:
        return row.publisher.get('name')
    except AttributeError:
        pass
    
def get_part(row):
    try:
        return row.Part_of.get('@type')[0]
    except AttributeError:
        pass

df0.type = df0.type.apply(lambda x: x if len(x)>2 else x[1])
df0.author = df0.apply(get_author, axis=1)
df0.location = df0.apply(get_location, axis=1)
df0.main_entity = df0.apply(get_entity, axis=1)
df0.publisher = df0.apply(get_publisher, axis=1)
df0.Part_of = df0.apply(get_part, axis=1)

df0.copyright_year = df0.copyright_year.fillna(0).astype(np.int64)
df0 = df0[df0.article_body.notna()]
df0 = df0.where(pd.notnull(df0), None) 

df0

,type,title,date_pub,date_mod,description,article_body,author,category,lang,location,copyright_year,main_entity,publisher,license,free_access,Part_of
0,ReportageNewsArticle,"Caro Quintero, the old drug lord who revolutio...",2022-07-17T23:16:53.566Z,2022-07-17T23:16:53.566Z,"At 69, he was the longest-serving drug traffic...",“Why are you smiling so much? Has life treated...,Jacobo García,International,en-us,Mexico,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
1,ReportageNewsArticle,Psychedelic therapy: Fighting depression with ...,2022-07-16T19:24:17.663Z,2022-07-16T19:24:17.663Z,‘Magic’ fungi are used for therapy around the ...,"When María del Mar Betancur opens her eyes, sh...",Noor Mahtani,International,en-us,Medellín,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
2,ReportageNewsArticle,The last orchid,2022-07-16T00:40:20.077Z,2022-07-16T00:40:20.077Z,A group of researchers in Colombia has discove...,"This past Thursday, a group of researchers fro...",Lucía Franco,International,en-us,Bogotá,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
3,ReportageNewsArticle,Rafael Caro Quintero: the historic head of the...,2022-07-15T23:38:17.996Z,2022-07-15T23:38:17.996Z,Sources from the Marines confirm that the vete...,"Rafael Caro Quintero, the legendary narco lead...",Elena Reina,International,en-us,Mexico,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
4,ReportageNewsArticle,Lobo: The surname that encapsulates all of Hon...,2022-07-15T16:05:24.186Z,2022-07-15T16:05:24.186Z,The execution-style killing of a son of ex-pre...,There are surnames that seem to encapsulate th...,Jacobo García,International,en-us,Mexico,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,ReportageNewsArticle,"US targets Putin’s daughters, leading Russian ...",2022-04-08T10:13:34.793Z,2022-04-08T10:13:34.793Z,Dramatic images from Bucha have led to additio...,There was still a lot of ammunition left in th...,Iker Seisdedos,USA,en-us,Washington,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
263,NewsArticle,Is Putin losing the war? The balance of the fi...,2022-04-08T02:10:18.594Z,2022-04-08T02:10:18.594Z,The Russian invasion has lost its impetus and ...,"On February 24, Russia attacked Ukraine in the...",Andrea Rizzi,International,en-us,None,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
264,ReportageNewsArticle,The scientist who fled Cuba and played a key r...,2022-04-07T17:58:37.044Z,2022-04-07T17:58:37.044Z,"Rolando Pajón, the company’s new medical direc...","In 2007, Noelia Álvarez, 82, was the only pers...",Pablo Linde,Science & Tech,en-us,Madrid,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork
265,ReportageNewsArticle,Nayib Bukele launches totalitarian crusade aga...,2022-04-07T15:44:33.548Z,2022-04-07T15:44:33.548Z,The popular president faces a huge security ch...,As if they were part of a perfectly coordinate...,Jacobo García,International,en-us,San Salvador,2022,WebPage,Ediciones EL PAÍS S.L.,https://elpais.com/estaticos/terms-and-conditi...,True,CreativeWork


In [12]:
## Part 3:  Use of psycopg2 to export the cleaned DataFrame into a Postgres table 
#         contained in a DataBase inside a T3.micro EC2 instance previously created.

import psycopg2
import psycopg2.extras as extras
import sys

params_dic = {# paste params
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(params_dic)

Connecting to the PostgreSQL database...
Connection successful


In [13]:
cur = conn.cursor()
# Drop table if exist
#cur.execute("DROP TABLE NEWS_EL_PAIS;")
cur.execute("""
                CREATE TABLE IF NOT EXISTS NEWS_EL_PAIS
                (id serial PRIMARY KEY,
                type varchar,
                title varchar,
                date_pub date,
                date_mod date,
                description varchar,
                article_body varchar,
                author varchar,
                category varchar,
                lang varchar,
                location varchar,
                copyright_year integer,
                main_entity varchar,
                publisher varchar,
                license varchar,
                free_access bool,
                Part_of varchar);
            """
           )
conn.commit()
cur.close()

In [14]:
def query_write_df(conn, df, table, **kwargs):
    """
    Use psycopg2.extras.execute_values() to insert a pandas dataframe into a Postgres Table
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query  = 'INSERT INTO %s(%s) VALUES %%s' % (table, cols)
    cursor = conn.cursor()
    # Clear data
    if(kwargs['ow']==True):
        cursor.execute('truncate %s RESTART IDENTITY;' % table)
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error: %s' % error)
        conn.rollback()
        cursor.close()
        return 1
    print('execute_values() done')
    cursor.close()

query_write_df(conn,df0,'NEWS_EL_PAIS',ow=True)

execute_values() done


In [15]:
# Checks the tables we have in our DB

cur = conn.cursor()
cur.execute("""
            SELECT table_name FROM information_schema.tables
            WHERE table_schema = 'public'
            """)
for table in cur.fetchall():
    print(table)
cur.close()

('news_el_pais',)


In [ ]:
# Done